# MONAI Auto3DSeg Hyper-parameter Optimization with NNI

This notebook provides an example to perform hype-parameter optimization(HPO) on learning rate with grid search method for spleen segmentation using NNI.

Note: if you have used other notebooks under `auto3dseg`, for examples: 
- `auto_runner.ipynb`
- `auto3dseg_autorunner_ref_api.ipynb`
- `auto3dseg_hello_world.ipynb`
- `hpo_optuna.ipynb`

You may have already generated the algorithm templates in MONAI bundle formats (hint: find them in the working directory). 

Please feel free to skip step 1-5 if the bundles are already generated.

## 1. Import libraries for HPO and pipelines

In [1]:
import os
import torch
import yaml

# download MSD dataset
from monai.apps import download_and_extract
from monai.apps.auto3dseg import BundleGen, DataAnalyzer, NNIGen
from monai.apps.auto3dseg.utils import export_bundle_algo_history, import_bundle_algo_history
from monai.bundle.config_parser import ConfigParser

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Define experiment file pathes

In [3]:
# Dataset pathes
data_root = "./"
msd_task = "Task09_Spleen"
dataroot = os.path.join(data_root, msd_task)

# User created files
datalist_file = os.path.join("..", "tasks", "msd", msd_task, "msd_" + msd_task.lower() + "_folds.json")
input_yaml = './input.yaml'
nni_yaml = './nni_config.yaml'

# Experiment setup
test_path = "./"
work_dir = os.path.join(test_path, "hpo_nni_work_dir")
datastats_file = os.path.join(work_dir, "datastats.yaml")
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

# Algorithm selected to do HPO. Refer to bundle history for the mapping between
# algorithm name and index, 0 is SegResNet2D
selected_algorithm_index = 0

## 3. Download one of MSD datasets

In [4]:
resource = "https://msd-for-monai.s3-us-west-2.amazonaws.com/" + msd_task + ".tar"
compressed_file = os.path.join(data_root, msd_task + ".tar")
if not os.path.exists(dataroot):
    os.makedirs(dataroot)
    download_and_extract(resource, compressed_file, data_root)

## 4. Generate input yaml and datafolds yaml. (User should generate their own)

In [5]:
input_dict = {
    "name": msd_task,
    "task": "segmentation",
    "modality": "MRI",
    "datalist": datalist_file,
    "dataroot": dataroot,
    "multigpu": True,
    "class_names": ["val_acc_pz", "val_acc_tz"]
}

with open(input_yaml, 'w') as f:
    yaml.dump(input_dict, f)


## 5. Create Bundle Generators


In [7]:
cfg = ConfigParser.load_config_file(input_yaml)
# data analysis
if not os.path.exists(datastats_file):
    da = DataAnalyzer(datalist_file, dataroot, output_path=datastats_file)
    da.get_all_case_stats()

# algorithm generation
bundle_generator = BundleGen(
    algo_path=work_dir,
    data_stats_filename=datastats_file,
    data_src_cfg_name=input_yaml,
)

bundle_generator.generate(work_dir, num_fold=5)
history = bundle_generator.get_history()
export_bundle_algo_history(history)

algo_templates.tar.gz: 100%|██████████| 280k/280k [00:00<00:00, 427kB/s]  

2022-09-21 04:46:15,948 - INFO - Downloaded: /tmp/tmpxnwm0v8d/algo_templates.tar.gz
2022-09-21 04:46:15,949 - INFO - Expected md5 is None, skip md5 check for file /tmp/tmpxnwm0v8d/algo_templates.tar.gz.
2022-09-21 04:46:15,949 - INFO - Writing into directory: ./workdir.


2022-09-21 04:46:16,499 - INFO - ./workdir/segresnet2d_0
2022-09-21 04:46:17,036 - INFO - ./workdir/segresnet2d_1
2022-09-21 04:46:17,473 - INFO - ./workdir/segresnet2d_2
2022-09-21 04:46:18,007 - INFO - ./workdir/segresnet2d_3
2022-09-21 04:46:18,452 - INFO - ./workdir/segresnet2d_4
2022-09-21 04:46:18,998 - INFO - ./workdir/dints_0
2022-09-21 04:46:19,446 - INFO - ./workdir/dints_1
2022-09-21 04:46:19,991 - INFO - ./workdir/dints_2
2022-09-21 04:46:20,441 - INFO - ./workdir/dints_3
2022-09-21 04:46:21,001 - INFO - ./workdir/dints_4
2022-09-21 04:46:21,438 - INFO - ./workdir/swinunetr_0
2022-09-21 04:46:21,992 - INFO - ./workdir/swinunetr_1
2022-09-21 04:46:22,430 - INFO - ./workdir/swinunetr_2
2022-09-21 04:46:22,866 - INFO - ./workdir/swinunetr_3
2022-09-21 04:46:23,438 - INFO - ./workdir/swinunetr_4
2022-09-21 04:46:23,998 - INFO - ./workdir/segresnet_0
2022-09-21 04:46:24,441 - INFO - ./workdir/segresnet_1
2022-09-21 04:46:25,004 - INFO - ./workdir/segresnet_2
2022-09-21 04:46:25,

## 6. Create Algo object from bundle_generator history

In [8]:
# you can get history from bundle_generator. It can also be acquired by reading bundles saved on disk
try:
    history = bundle_generator.get_history()
    assert len(history) > 0
except Exception:
    history = import_bundle_algo_history(work_dir, only_trained=False)

algo_dict = history[selected_algorithm_index]
algo_name = list(algo_dict.keys())[selected_algorithm_index]
algo = algo_dict[algo_name]

In [9]:
# "override_params" is used to update algorithm hyperparameters 
# like num_epochs, which are not in the HPO search space. We set num_epochs=2
# to shorten the training time as an example

max_epochs = 2

# safeguard to ensure max_epochs is greater or equal to 2
max_epochs = max(max_epochs, 2)

num_gpus = 1 if "multigpu" in input_dict and not input_dict["multigpu"] else torch.cuda.device_count()

num_epoch = max_epochs
num_images_per_batch = 2
n_data = 24  # total is 30 images, hold out one set (6 images) for cross fold val.
n_iter = int(num_epoch * n_data / num_images_per_batch / num_gpus)
n_iter_val = int(n_iter / 2)

override_param = {
    "num_iterations": n_iter,
    "num_iterations_per_validation": n_iter_val,
    "num_images_per_batch": num_images_per_batch,
    "num_epochs": num_epoch,
    "num_warmup_iterations": n_iter_val,
}

In [10]:
nni_gen = NNIGen(algo=algo, params=override_param)

2022-09-21 04:47:33,277 - INFO - ./workdir/segresnet2d_0_override
2022-09-21 04:47:33,277 - INFO - ============================================================================================================================================
2022-09-21 04:47:33,278 - INFO - If NNI will run in your local env: 
2022-09-21 04:47:33,278 - INFO - 1. Add the following line to the trialCommand in your NNI config: 
2022-09-21 04:47:33,278 - INFO - python -m monai.apps.auto3dseg NNIGen run_algo  ./workdir/segresnet2d_0_override/algo_object.pkl {result_dir}
2022-09-21 04:47:33,278 - INFO - --------------------------------------------------------------------------------------------------------------------------------------------
2022-09-21 04:47:33,279 - INFO - If NNI will run in a remote env: 
2022-09-21 04:47:33,279 - INFO - 1. Copy the algorithm_templates folder ./workdir/algorithm_templates/segresnet2d to remote {remote_algorithm_templates_dir}
2022-09-21 04:47:33,279 - INFO - 2. Copy the older

## 7. Create your NNI configs. Refer to [NNI](https://nni.readthedocs.io/en/stable/) for more details

In [11]:
nni_config = {
    "experimentName": msd_task + "_lr",
    "searchSpace": {
        "learning_rate": {
            "_type": "choice",
            "_value": [0.0001, 0.001, 0.01, 0.1]
        }
    },
    "trialCommand": None,
    "trialCodeDirectory": ".",
    "trialGpuNumber": 1,
    "trialConcurrency": 2,
    "maxTrialNumber": 10,
    "maxExperimentDuration": "1h",
    "tuner": {"name": "GridSearch"},
    "trainingService": {
        "platform": "local", "useActiveGpu": True}
}
with open(nni_yaml, 'w') as f:
    yaml.dump(nni_config, f)

## 8. Run NNI from terminal
### Step 1: copy the trialCommand print out info, e.g.
```
python -m monai.apps.auto3dseg NNIGen run_algo  ./workdir/segresnet2d_0/algo_object.pkl {result_dir}
```
Replace {result_dir} with a folder path to save HPO experiments.
### Step 2: copy the above trialCommand to replace the trialCommand in nni_config.yaml
### Step 3: run NNI experiemtns from a terminal with 
```
nnictl create --config ./nni_config.yaml
```

Use the print out trialCommand from NNIGen initialization to replace the trialCommand in nni_config and run NNI from terminal

## 9. Example Results
We changed override_param to {'num_iterations':6000, 'num_iterations_per_validation':600}, to run the experiments for longer time.
Here is the results shown in NNI webui. The optimal learning rate for SegResNet2D (selected_algorithm_index=0) is 0.1, which achieves Dice score of 0.735.

![](../figures/nni_image0.png)
![](../figures/nni_image1.png)
